In [21]:
#counting stars in the news.
#need function to add stars
#need prompt
#need function to generate stars
#need to get the correlation with the human annotations
import pandas as pd
import numpy as np
import json
from scipy.stats import kendalltau, spearmanr
import re
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,AutoModelForCausalLM
import torch

from openai import OpenAI          
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8880/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


In [35]:
#insert according to the average word number
def insert_star(text,star_list,star_number):
    words_list = re.findall(r'\S+|\n', text)
    # from nltk import word_tokenize
    # words_list = word_tokenize(text)
    interval = len(words_list) // star_number
    res = ''  
    for i in range(star_number):
        current_star = star_list[i]
        single_star = f"The little penguin counted {current_star} ★. "
        st = i*interval
        ed = (i+1)*interval
        if(i==star_number-1):
            ed = len(words_list)
        for j in range(st,ed):
            if(words_list[j] == '\n'):
                res = res.strip()
                res += words_list[j]
            else:
                res += words_list[j] + ' '
        res += single_star 
    res = res.strip()
    return res

def insert_star_after_sentence(text,star_list,star_number):
    from nltk.tokenize import sent_tokenize  
    sentences_list = sent_tokenize(text)
    # sentences_list = re.split(r'(?<=[.!?])\s+', text)
    interval = len(sentences_list) // star_number
    if(interval == 0):
        interval = 1
        star_number = len(sentences_list)
    res = ''
    for i in range(star_number):
        current_star = star_list[i]
        single_star = f"The little penguin counted {current_star} ★. "
        st = i*interval
        ed = (i+1)*interval
        if(i==star_number-1):
            ed = len(sentences_list)
        for j in range(st,ed):
            res += sentences_list[j] + ' '
        res += single_star
    res = res.strip()
    return res
            

def generate_star_list(star_number):
    return np.random.randint(1,20,star_number)

def LLM_count(path,star_number=2,llm_model="Qwen/Qwen1.5-7B",mode="sentence"):
    #temporal test
    test = "News: This a test example. I said \"who are you?\" I am Nick, my name is jack."\
            " Who are you? I want to eat some food, I am hungry. But I want to drink water! HHHHHH?"
    
    prompt = 'The little penguin is counting ★ in the news. '\
            'Please help little penguin collect all the numbers of ★, for example: {\"Little Penguin\":[x,x,x,...]}, no sum is required, '\
            'the numbers in [x,x,x,...] are the number of ★ the penguin counted each time. Please only output in JSON format without any explanation.'        
    
    star_list = generate_star_list(star_number)        
    print(f'star list: {star_list}')
    
    input_text = insert_star_after_sentence(test,star_list,star_number)
    # input_text = prompt+'\n'+input_text+'\nResult: '  
    print(input_text)        
    model = AutoModelForCausalLM.from_pretrained(llm_model, torch_dtype="auto", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(llm_model, trust_remote_code=True)
    with torch.no_grad():
        # l = len(input_text)
        # input_text = tokenizer(input_text, return_tensors="pt",return_attention_mask=False)
        # output = model.generate(**input_text, max_new_tokens=20)
        # print(tokenizer.batch_decode(output)[0][l:])
        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": input_text+'\nResult: '}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt")
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=50
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(response)
    
    #chat model
    # chat_response = client.chat.completions.create(
    #     model="01-ai/Yi-34B-Chat",
    #     messages=[
    #         {"role": "system", "content": prompt},
    #         {"role": "user", "content": "News: "+input_text+'\nResult: '},
            
    #     ],
    #     temperature=0,
    #     max_tokens=50,
    # )
    
    # res = chat_response.choices[0].message.content
    # print(res)    
        
    
    

LLM_count(1)


star list: [10  6]
News: This a test example. I said "who are you?" I am Nick, my name is jack. The little penguin counted 10 ★. Who are you? I want to eat some food, I am hungry. But I want to drink water! HHHHHH? The little penguin counted 6 ★.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{
"Little Penguin": [10, 6]
}
system
You are a helpful assistant.
user
System: You are an AI assistant. You will be given a task. You must generate a detailed and


这是一个示例文本. 这是第二个句子. 这是第三个句子.*** 这是第四个句子. 这是第五个句子.*** 这是第六个句子。
